In [ ]:

import pandas as pd
from sqlalchemy import create_engine, text

In [ ]:

def get_dataCR():
    DB_HOST = "37.61.241.45"
    DB_PORT = "5432"
    DB_NAME = "Khubeo_IA"
    DB_USER = "postgres"
    DB_PASS = "Xjp2yCm$G36WR4E"
    
    engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
    
    with engine.connect() as conn:
        result = conn.execute(text("""select * from t_compte_rendu cr
        left join t_dates da
            on da.fulldatealternatekey = cr.cr_date
        left join t_dossier dos
            on dos.id_dossier = cr.id_dossier
        left join t_enseigne en
            on en.id_enseigne = dos.do_id_enseigne
    where cr.id_client not in  (15,31,100, 101) limit 500;"""))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())  # Convert result to DataFrame
        return df
df = get_dataCR()

print("Loaded dataframe with {df.shape[0]} rows and {df.shape[1]} columns")

df.head()

# Print column names as a list
print(df.columns.tolist())


In [ ]:
def compute_hebergement_service(row):
    """
    Process financial data for accommodation and services, calculating TTC, HT, and TVA values.
    Also calculates total values across accommodation and services.
    """
    # Helper function to safely convert to float (returns original value if conversion fails)
    def safe_float(value):
        # Try to convert to float, return original value if it fails
        try:
            return float(value)  # Attempt to convert the value to a float
        except (TypeError, ValueError):  # Handle cases where conversion isn't possible
            return value  # Return the original value if conversion fails
    
    # Helper function to calculate the three financial values (TTC, HT, TVA)
    def calculate_financial_values(ttc, ht, tva):
        # Calculate TTC (price with tax) if missing or zero (using HT * 1.10)
        if pd.isnull(ttc) or ttc == 0:  # Check if TTC is null or zero
            ttc = ht * 1.10 if pd.notnull(ht) and ht != 0 else ttc  # Apply 10% tax to HT if available
        
        # Calculate HT (price without tax) if missing or zero (using TTC / 1.10)
        if pd.isnull(ht) or ht == 0:  # Check if HT is null or zero
            ht = ttc / 1.10 if pd.notnull(ttc) and ttc != 0 else ht  # Derive HT from TTC if available
        
        # Calculate TVA (tax amount) if missing or zero
        if pd.isnull(tva) or tva == 0:  # Check if TVA is null or zero
            if pd.notnull(ht) and ht > 0:
                tva = ht * 0.10  # Calculate TVA as 10% of HT if HT is available
            elif pd.notnull(ttc) and ttc != 0:
                tva = ttc - (ttc / 1.10)  # Calculate TVA by subtracting HT from TTC
            # else: keep original tva value
        
        return ttc, ht, tva  # Return the calculated or original values
    
    # Extract and convert accommodation values from the input row
    hebergement_ttc = safe_float(row['cr_hebergement_ttc'])  # Get accommodation price with tax
    hebergement_ht = safe_float(row['cr_hebergement_ht'])  # Get accommodation price without tax
    hebergement_tva = safe_float(row['cr_hebergement_tva'])  # Get accommodation tax amount
    
    # Calculate complete accommodation financial values using the helper function
    ttc_hebergement, ht_hebergement, tva_hebergement = calculate_financial_values(
        hebergement_ttc, hebergement_ht, hebergement_tva
    )
    
    # Extract and convert service values from the input row
    service_ttc = safe_float(row['cr_service_ttc'])  # Get service price with tax
    service_ht = safe_float(row['cr_service_ht'])  # Get service price without tax
    service_tva = safe_float(row['cr_service_tva'])  # Get service tax amount
    
    # Calculate complete service financial values using the helper function
    ttc_service, ht_service, tva_service = calculate_financial_values(
        service_ttc, service_ht, service_tva
    )
    
    # Calculate total HT (sum of accommodation and service HT)
    total_ht = None  # Initialize total HT as None
    if pd.notnull(ht_hebergement) or pd.notnull(ht_service):  # Check if either HT value is available
        total_ht = (ht_hebergement if pd.notnull(ht_hebergement) else 0) + (ht_service if pd.notnull(ht_service) else 0)
    
    # Calculate total TTC (sum of accommodation and service TTC)
    total_ttc = None  # Initialize total TTC as None
    if pd.notnull(ttc_hebergement) or pd.notnull(ttc_service):  # Check if either TTC value is available
        total_ttc = (ttc_hebergement if pd.notnull(ttc_hebergement) else 0) + (ttc_service if pd.notnull(ttc_service) else 0)
    
    # Calculate total TVA (sum of accommodation and service TVA)
    total_tva = None  # Initialize total TVA as None
    if pd.notnull(tva_hebergement) or pd.notnull(tva_service):  # Check if either TVA value is available
        total_tva = (tva_hebergement if pd.notnull(tva_hebergement) else 0) + (tva_service if pd.notnull(tva_service) else 0)
    
    # Return all calculated values as a pandas Series with named columns
    return pd.Series({
        'cr_chb_dispo': row['cr_chb_dispo'],  # Pass through available rooms
        'cr_chb_louees': row['cr_chb_louees'],  # Pass through rented rooms
        'cr_hebergement_ttc': ttc_hebergement,  # Include calculated accommodation TTC
        'cr_hebergement_ht': ht_hebergement,  # Include calculated accommodation HT
        'cr_hebergement_tva': tva_hebergement,  # Include calculated accommodation TVA
        'cr_service_ttc': ttc_service,  # Include calculated service TTC
        'cr_service_ht': ht_service,  # Include calculated service HT
        'cr_service_tva': tva_service,  # Include calculated service TVA
        'cr_total_ttc': total_ttc,  # Include calculated total TTC
        'cr_total_ht': total_ht,  # Include calculated total HT
        'cr_total_tva': total_tva  # Include calculated total TVA
    })

# Apply the function to each row in the dataframe to calculate all values
# Apply method is used to apply a function along the axis of the DataFrame
# axis=1 applies the function to each row, axis=0 would apply it to each column
# returns a new DataFrame with the results of the function applied to each row
# First get the data if not already retrieved
if 'df' not in locals():
    df = get_dataCR()

# Apply the function to each row in the dataframe to calculate values
calculated_columns = df.apply(compute_hebergement_service, axis=1)

# Create a new DataFrame with all columns - replace only the calculated columns
df_cr_verified = df.copy()
for col in calculated_columns.columns:
    df_cr_verified[col] = calculated_columns[col]

# Display the resulting DataFrame with all calculated values
print(df_cr_verified.head())



In [ ]:

# Check if the column names are identical in the original and processed dataframes
# set is used to compare the unique values in each set of columns
original_columns = set(df.columns)
processed_columns = set(df_hebergement_service.columns)

# Check if the sets of columns are identical
columns_are_same = original_columns == processed_columns

# Display the result
print(f"Column names are identical: {columns_are_same}")

# If they're not the same, show the differences
if not columns_are_same:
    print("Columns only in original df:", original_columns - processed_columns)
    print("Columns only in processed df:", processed_columns - original_columns)
else:
    print("Both dataframes have exactly the same columns.")


